In [1]:
import glob
from natsort import natsorted
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from typing import Final
from __future__ import annotations

In [2]:
PLAYER_RESULTS_HEADER: Final[tuple[str]] = (
    '選手名','年齢','性別','身長','体重','級班','脚質','登録番号',
    'レース名','着順1','着順2','着順3','着順4','着順5'
)
PLAYER_RESULTS_HEADER_RENAME: Final[tuple[str]] = (
    '選手名','性別','級班','脚質','登録番号',
    '年齢','身長','体重','着順1','着順2','着順3','着順4','着順5','レース開始日','レース名','グレード'
)
PLAYER_RESULTS_CONV_HEADER: Final[dict(str, int)] = {
    '年齢': 1,
    '身長': 3,
    '体重': 4,
    '着順1': 9,
    '着順2': 10,
    '着順3': 11,
    '着順4': 12,
    '着順5': 13,
}
DROP_COLS: Final[list[str]] = [
    '性別','級班','脚質','登録番号','年齢','身長','体重'
]
PLAYER_RESULTS_REINDEX_COLS: Final[tuple[str]] = (
    '選手名','レース開始日','レース名','グレード','着順1','着順2','着順3','着順4','着順5'
)
LABELENCODER_COLUMNS: Final[list[str]] = ['性別', '級班', '脚質', 'グレード']

In [3]:
def get_df() -> pd.DataFrame:
    files = natsorted(glob.glob('player_data/player_results*.csv'))

    player_results_list = []
    for file in files:
        try:
            player_results_list.append(pd.read_csv(file, encoding='cp932', header=0, names=PLAYER_RESULTS_HEADER))
        except UnicodeDecodeError:
            player_results_list.append(pd.read_csv(file, encoding='utf-8', header=0, names=PLAYER_RESULTS_HEADER))
    df_player_results = pd.concat(player_results_list)
    
    return df_player_results

In [4]:
def char_exclusion(df: pd.DataFrame) -> pd.DataFrame:
    for column_name, column_num in PLAYER_RESULTS_CONV_HEADER.items():
        if column_num == 1 or column_num == 9 or column_num == 10 or \
           column_num == 11 or column_num == 12 or column_num == 13:
            df[column_name + '(数字)'] = df.iloc[:, column_num].str.extract('(\d+)')
        elif column_num == 3:
            df[column_name + '(数字)'] = df.iloc[:, column_num].str.rstrip('cm')
        else:
            df[column_name + '(数字)'] = df.iloc[:, column_num].str.rstrip('kg')
    return df

In [5]:
def df_drop(df: pd.DataFrame, column_names: list[str]) -> pd.DataFrame:
    for column_name in column_names:
        df.drop(column_name, axis=1, inplace=True)
    return df

In [6]:
def race_name_split(df: pd.DataFrame) -> pd.DataFrame:
    # レース名で半角が混在しているため、全角に置換
    df['レース名'] = df['レース名'].replace('21/5/31～6/2 静岡CTC杯FI', '21/5/31～6/2 静岡ＣＴＣ杯FI')
    df_race_name_split = df['レース名'].str.extract(
        '(?P<レース開始日>^.+)～.+ (?P<レース名単独>[^\x01-\x7E]+)(?P<グレード>[GFIP]+)')
    df = pd.concat([df, df_race_name_split], axis=1)
    df.drop('レース名', axis=1, inplace=True)
    return df

In [7]:
def labelencoder(df: pd.DataFrame) -> pd.DataFrame:
    for column in LABELENCODER_COLUMNS:
        le = LabelEncoder()
        le.fit(df[column])
        df[column] = le.transform(df[column])

    return df

In [8]:
df_player_results: pd.DataFrame = get_df()
df_player_results = char_exclusion(df_player_results)
df_player_results = df_drop(df_player_results, list(PLAYER_RESULTS_CONV_HEADER.keys()))
df_player_results = race_name_split(df_player_results)
df_player_results['レース開始日'] = pd.to_datetime(df_player_results['レース開始日'], format='%y/%m/%d')
# df_player_results = labelencoder(df_player_results)
df_player_results.columns = PLAYER_RESULTS_HEADER_RENAME
df_player_results.drop(DROP_COLS, axis=1, inplace=True)
df_player_results = df_player_results.reindex(columns=PLAYER_RESULTS_REINDEX_COLS)
pd.to_pickle(df_player_results, 'player_results.pkl')
df_player_results = pd.read_pickle('player_results.pkl')
df_player_results

,選手名,着順1,着順2,着順3,着順4,着順5,レース開始日,レース名,グレード
0,相川永伍,NaN,NaN,NaN,NaN,NaN,2021-07-08,福井不死鳥杯,GIII
1,相川永伍,1,6,3,NaN,NaN,2021-06-18,取手坂巻正巳杯・日刊スポーツ賞,FI
2,相川永伍,3,4,2,5,NaN,2021-06-10,松山国際自転車トラック競技支援競輪,GIII
3,相川永伍,1,3,6,NaN,NaN,2021-06-01,京王閣Ｓ市原カップ＆デイリー杯,FI
4,相川永伍,5,1,3,NaN,NaN,2021-05-25,向日町ムコリン・ムッチー賞,FI
...,...,...,...,...,...,...,...,...,...
904,鰐渕正利,7,3,NaN,NaN,NaN,2012-07-25,豊橋中日スポーツ昇龍杯争奪戦,FI
905,鰐渕正利,7,4,8,NaN,NaN,2012-07-17,伊東チャリロト杯,FI
906,鰐渕正利,8,2,8,2,NaN,2012-07-05,小松島阿波おどり杯争覇戦,GIII
907,鰐渕正利,4,8,7,NaN,NaN,2012-06-26,青森Ｊｏｍｏｎ ＧＰ,FI


# 試行

In [9]:
df_player_results.set_index('選手名')

,着順1,着順2,着順3,着順4,着順5,レース開始日,レース名,グレード
選手名,,,,,,,,
相川永伍,NaN,NaN,NaN,NaN,NaN,2021-07-08,福井不死鳥杯,GIII
相川永伍,1,6,3,NaN,NaN,2021-06-18,取手坂巻正巳杯・日刊スポーツ賞,FI
相川永伍,3,4,2,5,NaN,2021-06-10,松山国際自転車トラック競技支援競輪,GIII
相川永伍,1,3,6,NaN,NaN,2021-06-01,京王閣Ｓ市原カップ＆デイリー杯,FI
相川永伍,5,1,3,NaN,NaN,2021-05-25,向日町ムコリン・ムッチー賞,FI
...,...,...,...,...,...,...,...,...
鰐渕正利,7,3,NaN,NaN,NaN,2012-07-25,豊橋中日スポーツ昇龍杯争奪戦,FI
鰐渕正利,7,4,8,NaN,NaN,2012-07-17,伊東チャリロト杯,FI
鰐渕正利,8,2,8,2,NaN,2012-07-05,小松島阿波おどり杯争覇戦,GIII


In [10]:
from datetime import datetime
t_date = datetime(2018, 12, 15)
t_date
names=['鰐渕正利', '相川永伍']

In [11]:
# df = df_player_results[(df_player_results['選手名'] == names) & (df_player_results['レース開始日'] <= t_date)].head()

In [12]:
df = df_player_results.query("選手名 == @names & レース開始日 <= @t_date")
df

,選手名,着順1,着順2,着順3,着順4,着順5,レース開始日,レース名,グレード
71,相川永伍,9,6,6,8,NaN,2018-12-13,伊東椿賞争奪戦,GIII
72,相川永伍,5,1,7,7,NaN,2018-12-06,広島ひろしまピースカップ,GIII
73,相川永伍,7,NaN,NaN,NaN,NaN,2018-11-20,西武園ＫＥＩＲＩＮフェスティバル,FI
74,相川永伍,7,4,4,NaN,NaN,2018-11-17,小田原神奈川新聞社杯,FI
75,相川永伍,8,4,5,NaN,NaN,2018-11-05,福井スポニチ杯 福井チャンピオンＲ,FI
...,...,...,...,...,...,...,...,...,...
904,鰐渕正利,7,3,NaN,NaN,NaN,2012-07-25,豊橋中日スポーツ昇龍杯争奪戦,FI
905,鰐渕正利,7,4,8,NaN,NaN,2012-07-17,伊東チャリロト杯,FI
906,鰐渕正利,8,2,8,2,NaN,2012-07-05,小松島阿波おどり杯争覇戦,GIII
907,鰐渕正利,4,8,7,NaN,NaN,2012-06-26,青森Ｊｏｍｏｎ ＧＰ,FI


In [13]:
df = pd.DataFrame()
for name in names:
    temp_df = df_player_results.query("選手名 == @name and レース開始日 <= @t_date").head()
    df = pd.concat([df, temp_df])
df

,選手名,着順1,着順2,着順3,着順4,着順5,レース開始日,レース名,グレード
740,鰐渕正利,5,6,7,NaN,NaN,2018-12-02,小倉スポーツ報知杯,FI
741,鰐渕正利,3,8,6,NaN,NaN,2018-11-17,京王閣ＪＦＮ杯＆東京スポーツ杯,FII
742,鰐渕正利,4,6,7,NaN,NaN,2018-10-31,向日町ムコリン・ムッチー賞,FI
743,鰐渕正利,4,7,4,NaN,NaN,2018-10-19,富山オール７ケイリン,FII
744,鰐渕正利,4,9,9,NaN,NaN,2018-10-01,豊橋三遠ネオフェニックス杯争奪戦,FI
71,相川永伍,9,6,6,8,NaN,2018-12-13,伊東椿賞争奪戦,GIII
72,相川永伍,5,1,7,7,NaN,2018-12-06,広島ひろしまピースカップ,GIII
73,相川永伍,7,NaN,NaN,NaN,NaN,2018-11-20,西武園ＫＥＩＲＩＮフェスティバル,FI
74,相川永伍,7,4,4,NaN,NaN,2018-11-17,小田原神奈川新聞社杯,FI
75,相川永伍,8,4,5,NaN,NaN,2018-11-05,福井スポニチ杯 福井チャンピオンＲ,FI


In [14]:
df_list = []
for name in names:
    temp_df = df.query("選手名 == @name")
    temp_base = temp_df.iloc[[0], 0:6]
    for i in range(len(temp_df)):
        temp_results = temp_df.iloc[[i], [7, 8, 9, 10, 11, 14]]
        temp_base = pd.concat([temp_base, temp_results], axis=1)
    df_list.append(temp_base)

IndexError: positional indexers are out-of-bounds

In [ ]:
df2 = pd.concat(df_list)
df2

,選手名,性別,級班,脚質,登録番号,年齢,体重,着順1,着順2,着順3,...,着順2,着順3,着順4,レース名,体重,着順1,着順2,着順3,着順4,レース名
740,鰐渕正利,1.0,1.0,1.0,12208.0,51,76.0,5,6,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7,4,NaN,富山オール７ケイリン,NaN,NaN,NaN,NaN,NaN,NaN
744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,76.0,4,9,9,NaN,豊橋三遠ネオフェニックス杯争奪戦
71,相川永伍,1.0,5.0,2.0,14448.0,37,93.0,9,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,4,NaN,小田原神奈川新聞社杯,NaN,NaN,NaN,NaN,NaN,NaN
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,93.0,8,4,5,NaN,福井スポニチ杯 福井チャンピオンＲ
